In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [2]:
# Load the Sogou News dataset
raw_datasets = load_dataset("community-datasets/sogou_news")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/work/LAS/jannesar-lab/.ondemand/dphuong/jupyter-envs/20230427/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for community-datasets/sogou_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/community-datasets/sogou_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['title', 'content', 'label'],
        num_rows: 60000
    })
    train: Dataset({
        features: ['title', 'content', 'label'],
        num_rows: 450000
    })
})

In [8]:
raw_train_dataset = raw_datasets["train"]
raw_test_dataset = raw_datasets["test"]
raw_train_dataset[0]

{'title': '2008 di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha3n me3i nv3 mo2 te4 ',
 'content': '2008di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha3n yu2 15 ri4 za4i qi1ng da3o guo2 ji4 hui4 zha3n zho1ng xi1n she4ng da4 ka1i mu4 . be3n ci4 che1 zha3n jia1ng chi2 xu4 da4o be3n yue4 19 ri4 . ji1n nia2n qi1ng da3o guo2 ji4 che1 zha3n shi4 li4 nia2n da3o che2ng che1 zha3n gui1 mo2 zui4 da4 di2 yi1 ci4 , shi3 yo4ng lia3o qi1ng da3o guo2 ji4 hui4 zha3n zho1ng xi1n di2 qua2n bu4 shi4 ne4i wa4i zha3n gua3n . yi3 xia4 we2i xia4n cha3ng mo2 te4 tu2 pia4n .',
 'label': 3}

In [9]:
# Get unique titles in the train and test datasets
unique_train_titles = set(raw_train_dataset['label'])
unique_test_titles = set(raw_test_dataset['label'])

num_unique_train_titles = len(unique_train_titles)
num_unique_test_titles = len(unique_test_titles)

print(f"Number of unique titles in the training dataset: {num_unique_train_titles}")
print(f"Number of unique titles in the test dataset: {num_unique_test_titles}")

Number of unique titles in the training dataset: 5
Number of unique titles in the test dataset: 5


In [10]:
def tokenize_function(example):
    return tokenizer(example["content"], truncation=True)

In [11]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['title', 'content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60000
    })
    train: Dataset({
        features: ['title', 'content', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 450000
    })
})

# Prepare for training

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(["content", "title"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [15]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'token_type_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.6444, grad_fn=<NllLossBackward0>) torch.Size([8, 5])


In [18]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/work/LAS/jannesar-lab/.ondemand/dphuong/jupyter-envs/20230427/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

56250


# Training loop

In [23]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/56250 [00:00<?, ?it/s]

In [25]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"].cpu())

metric.compute()

{'accuracy': 0.33675}